In [13]:
import tensorflow as tf
import numpy as np
from pathlib import Path

In [2]:
DATA_PATH = Path("/scratch/ajb5d/ecg/tfrecords/")
TRAIN_RECS = list(DATA_PATH.glob("train*.tfrecords"))
VAL_RECS = list(DATA_PATH.glob("val*.tfrecords"))

In [3]:
BATCH_SIZE = 64

record_format = {
    'ecg/data': tf.io.FixedLenSequenceFeature([], tf.float32, allow_missing=True),
    'age': tf.io.FixedLenFeature([], tf.float32),
    'gender': tf.io.FixedLenFeature([], tf.int64),
}

def _parse_record(record):
    example = tf.io.parse_single_example(record, record_format)
    ecg_data = tf.reshape(example['ecg/data'], [5000,12])
    label = example['age']
    return ecg_data, label

def drop_na_ages(x,y):
    return not tf.math.reduce_any(tf.math.is_nan(y))

def age_lt_90(x,y):
    return tf.math.reduce_all(tf.math.less_equal(y, tf.constant([90.0])))

def load_dataset(filenames):
    ignore_order = tf.data.Options()
    ignore_order.experimental_deterministic = False
    dataset = tf.data.TFRecordDataset(filenames)
    dataset = dataset.with_options(ignore_order)
    dataset = dataset.map(_parse_record, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.filter(drop_na_ages)
    dataset = dataset.filter(age_lt_90)
    return dataset

def get_dataset(filenames, labeled=True):
    dataset = load_dataset(filenames)
    dataset = dataset.shuffle(2048)
    dataset = dataset.prefetch(buffer_size=tf.data.AUTOTUNE)
    dataset = dataset.batch(BATCH_SIZE)
    return dataset

In [4]:
train_dataset = get_dataset(TRAIN_RECS)
val_dataset = get_dataset(VAL_RECS)

2023-11-29 09:25:25.832283: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1636] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 78823 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:87:00.0, compute capability: 8.0


In [6]:
def conv_block(x, n, k, mp, groups = 12):
    x = tf.keras.layers.Conv1D(n * groups, k, padding='same', groups = groups)(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    if mp:
        x = tf.keras.layers.MaxPooling1D(mp)(x)

    return x

input_layer = tf.keras.layers.Input(shape=(5000, 12))
x = input_layer

x = conv_block(x, 16, 7, 2)
x = conv_block(x, 16, 5, 4)
x = conv_block(x, 32, 5, 2)
x = conv_block(x, 32, 5, 4)
x = conv_block(x, 64, 5, 2)
x = conv_block(x, 64, 3, 2)
x = conv_block(x, 64, 3, 2)
x = conv_block(x, 64, 3, 2)

x = tf.keras.layers.Conv1D(128, 12, padding = 'same', data_format = 'channels_first')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.MaxPooling1D()(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(128)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64)(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Activation('relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1)(x)

model = tf.keras.models.Model(input_layer, x)

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.MeanSquaredError(),
    metrics=['mse', 'mae']
)

In [ ]:
from datetime import datetime
import os

def make_checkpoint_dir(data_path, label):
    current_datetime = datetime.now()
    formatted_datetime = current_datetime.strftime("%Y-%m-%d_%H-%M-%S")
    output_dir = f"{label}-{formatted_datetime}"
    output_path = f"{data_path}/{output_dir}"
    
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    
    return output_path

model_name = "cnn-age"
output_path = make_checkpoint_dir("data/models", model_name)

print(f"Model: {model_name} Run Path: {output_path}")

In [8]:
callbacks = [
    tf.keras.callbacks.TerminateOnNaN(),
    tf.keras.callbacks.ReduceLROnPlateau(),
    tf.keras.callbacks.ModelCheckpoint(output_path, save_best=True),
    tf.keras.callbacks.CSVLogger(f"data/models/{model_name}-history.csv")
]

history = model.fit(train_dataset, epochs=50, validation_data=val_dataset, callbacks=callbacks)

Epoch 1/2


2023-11-29 09:25:36.567187: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55cdc5cd6a60 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-11-29 09:25:36.567588: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA A100-SXM4-80GB, Compute Capability 8.0
2023-11-29 09:25:36.671012: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8904
2023-11-29 09:25:38.211728: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-11-29 09:25:40.031814: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:625] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-11-29 09:25:40.205708: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:255] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to en

   7479/Unknown - 685s 89ms/step - loss: 415.8427 - mse: 415.8427 - mae: 14.8430INFO:tensorflow:Assets written to: data/models/cnn-age-2023-11-29_09-25-29/assets


INFO:tensorflow:Assets written to: data/models/cnn-age-2023-11-29_09-25-29/assets


7479/7479 [==============================] - 730s 95ms/step - loss: 415.8427 - mse: 415.8427 - mae: 14.8430 - val_loss: 161.0133 - val_mse: 161.0133 - val_mae: 10.2466 - lr: 0.0010
Epoch 2/2
7479/7479 [==============================] - ETA: 0s - loss: 188.4316 - mse: 188.4316 - mae: 10.9230INFO:tensorflow:Assets written to: data/models/cnn-age-2023-11-29_09-25-29/assets


INFO:tensorflow:Assets written to: data/models/cnn-age-2023-11-29_09-25-29/assets


7479/7479 [==============================] - 709s 95ms/step - loss: 188.4316 - mse: 188.4316 - mae: 10.9230 - val_loss: 134.0525 - val_mse: 134.0525 - val_mae: 9.0620 - lr: 0.0010


In [9]:
model.save(f"data/models/{model_name}.keras")